# Shallow Moment Tutorial (Simple)

## Imports

In [1]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false

import os
import numpy as np
import jax
from jax import numpy as jnp
import pytest
from types import SimpleNamespace
from sympy import cos, pi, Piecewise

from library.fvm.solver import HyperbolicSolver, Settings
from library.fvm.ode import RK1
import library.fvm.reconstruction as recon
import library.fvm.timestepping as timestepping
import library.fvm.flux as flux
import library.fvm.nonconservative_flux as nc_flux
from library.model.boundary_conditions import BoundaryCondition
from library.model.models.basisfunctions import Basisfunction, Legendre_shifted
from library.model.models.basismatrices import Basismatrices
from library.misc.misc import Zstruct

from library.model.models.shallow_moments import ShallowMoments2d
import library.model.initial_conditions as IC
import library.model.boundary_conditions as BC
import library.misc.io as io
from library.mesh.mesh import compute_derivatives
from tests.pdesoft import plots_paper
import library.postprocessing.visualization as visu


import library.mesh.mesh as petscMesh
import library.postprocessing.postprocessing as postprocessing
from library.mesh.mesh import convert_mesh_to_jax
import argparse

2025-08-28 21:38:13.882 | WARNING  | library.fvm.solver:<module>:28 - libprecice.so.3: cannot open shared object file: No such file or directory


## Model

In [2]:
level = 0
offset = 1+level
n_fields = 3 + 2 * level
settings = Settings(
    name="SME",
    output=Zstruct(
        directory=f"outputs/junction_{level}", filename="SME", output_snapshots=30
    ),
)

2025-08-28 21:38:26.754 | WARNING  | library.misc.misc:__init__:146 - No 'clean_directory' attribute found in output Zstruct. Default: False


In [4]:
inflow_dict = { 
    0: lambda t, x, dx, q, qaux, p, n: Piecewise((0.1, t < 0.2),(q[0], True)),
    1: lambda t, x, dx, q, qaux, p, n: Piecewise((-0.3, t < 0.2),(-q[1], True)),
                }
inflow_dict.update({
    1+i: lambda t, x, dx, q, qaux, p, n: 0 for i in range(level)
})
inflow_dict.update({
    1+offset+i: lambda t, x, dx, q, qaux, p, n: 0 for i in range(level+1)
})

bcs = BC.BoundaryConditions(
    [
        BC.Lambda(physical_tag="inflow", prescribe_fields=inflow_dict),
        BC.Wall(physical_tag="wall"),
    ]
)

def custom_ic(x):
    Q = np.zeros(3 + 2 * level, dtype=float)
    Q[0] = 0.01
    return Q

ic = IC.UserFunction(custom_ic)

model = ShallowMoments2d(
    level=level,
    boundary_conditions=bcs,
    initial_conditions=ic,
)

main_dir = os.getenv("ZOOMY_DIR")
mesh = petscMesh.Mesh.from_gmsh(
    os.path.join(main_dir, "meshes/channel_junction/mesh_2d_coarse.msh")
    # os.path.join(main_dir, "meshes/channel_junction/mesh_2d_fine.msh")
)

mesh = convert_mesh_to_jax(mesh)
class SMESolver(HyperbolicSolver):
    def update_qaux(self, Q, Qaux, Qold, Qauxold, mesh, model, parameters, time, dt):
        dudx = compute_derivatives(Q[1]/Q[0], mesh, derivatives_multi_index=[[0, 0]])[:,0]
        dvdy = compute_derivatives(Q[1+offset]/Q[0], mesh, derivatives_multi_index=[[0, 1]])[:,0]
        Qaux = Qaux.at[0].set(dudx)
        Qaux = Qaux.at[1].set(dvdy)
        return Qaux
solver = SMESolver(settings=settings)

Time compute matrices: 0.09297966957092285


## Solve

In [5]:
Qnew, Qaux = solver.solve(mesh, model)

2025-08-28 21:40:27.171 | INFO     | library.fvm.solver:log_callback_hyperbolic:44 - iteration: 10, time: 0.005779, dt: 0.000266, next write at time: 0.011111
2025-08-28 21:40:27.676 | INFO     | library.fvm.solver:log_callback_hyperbolic:44 - iteration: 20, time: 0.008436, dt: 0.000266, next write at time: 0.011111
2025-08-28 21:40:28.124 | INFO     | library.fvm.solver:log_callback_hyperbolic:44 - iteration: 30, time: 0.011093, dt: 0.000266, next write at time: 0.011111
2025-08-28 21:40:28.670 | INFO     | library.fvm.solver:log_callback_hyperbolic:44 - iteration: 40, time: 0.013750, dt: 0.000266, next write at time: 0.022222
2025-08-28 21:40:29.153 | INFO     | library.fvm.solver:log_callback_hyperbolic:44 - iteration: 50, time: 0.016406, dt: 0.000266, next write at time: 0.022222
2025-08-28 21:40:29.636 | INFO     | library.fvm.solver:log_callback_hyperbolic:44 - iteration: 60, time: 0.019064, dt: 0.000266, next write at time: 0.022222
2025-08-28 21:40:30.150 | INFO     | library.f

## Visualization

In [6]:
io.generate_vtk(os.path.join(settings.output.directory, f"{settings.output.filename}.h5"))
postprocessing.vtk_interpolate_3d(model, settings, Nz=20, filename='out_3d')

ValueError: could not broadcast input array from shape (16000,5) into shape (16000,6)

In [ ]:
visu.pyvista_3d(settings.output.directory, scale=1.0)

BokehModel(combine_events=True, render_bundle={'docs_json': {'27d40e6f-9809-46fc-aaa7-e2c427429dcd': {'version…